### Installing Atomate2 from source with OpenMM

```bash
# setting up our conda environment
>>> conda create -n atomate2 python=3.11
>>> conda activate atomate2
>>> git
# installing atomate2
>>> git clone https://github.com/orionarcher/atomate2.git
>>> cd atomate2
>>> git branch openff
>>> git checkout openff
>>> git pull origin openff
>>> pip install -e .
# installing classical_md dependencies
>>> conda install -c conda-forge --file .github/classical_md_requirements.txt
```


### Testing installation
```bash
# make sure that tests are passing for CUDA
>>> python -m openmm.testInstallation
```

In [1]:
from atomate2.classical_md.core import generate_interchange
import numpy as np
from jobflow import run_locally, Flow
from atomate2.classical_md.openmm.flows.core import AnnealMaker, ProductionMaker
from atomate2.classical_md.openmm.jobs.core import (
    EnergyMinimizationMaker,
    NPTMaker,
    NVTMaker,
)
from pymatgen.core.structure import Molecule


def run_job(job):
    response_dict = run_locally(job, ensure_success=True)
    return list(response_dict.values())[-1][1].output

In [ ]:
mol_specs_dicts = [
    {"smile": "O", "count": 200, "name": "water"},
    {"smile": "CCO", "count": 10, "name": "ethanol"},
    {"smile": "C1=C(C=C(C(=C1O)O)O)C(=O)O", "count": 1, "name": "gallic_acid"},
]

gallic_interchange_job = generate_interchange(mol_specs_dicts, 1.3)

In [2]:
pf6 = Molecule(
    ["P", "F", "F", "F", "F", "F", "F"],
    [
        [0.0, 0.0, 0.0],
        [1.6, 0.0, 0.0],
        [-1.6, 0.0, 0.0],
        [0.0, 1.6, 0.0],
        [0.0, -1.6, 0.0],
        [0.0, 0.0, 1.6],
        [0.0, 0.0, -1.6]
    ]
)
pf6_charges = np.array([1.34, -0.39, -0.39, -0.39, -0.39, -0.39, -0.39])

mol_specs_dicts = [
    {"smile": "C1COC(=O)O1", "count": 100, "name": "EC"},
    {"smile": "CCOC(=O)OC", "count": 100, "name": "EMC"},
    {
        "smile": "F[P-](F)(F)(F)(F)F",
        "count": 50,
        "name": "PF6",
        "partial_charges": pf6_charges,
        "geometry": pf6,
        "charge_scaling": 0.8,
        "charge_method": "RESP",
    },
    {"smile": "[Li+]", "count": 50, "name": "Li", "charge_scaling": 0.8},
]

elyte_interchange_job = generate_interchange(mol_specs_dicts, 1.3)

In [3]:

production_maker = ProductionMaker(
    name="test_production",
    energy_maker=EnergyMinimizationMaker(
        platform_name="CUDA",
        platform_properties={"DeviceIndex": "0"},
        # platform_name="CPU",
    ),
    npt_maker=NPTMaker(n_steps=100),
    anneal_maker=AnnealMaker.from_temps_and_steps(n_steps=150),
    nvt_maker=NVTMaker(n_steps=1000),
)

production_flow = production_maker.make(
    elyte_interchange_job.output.interchange,
    prev_task=elyte_interchange_job.output,
    output_dir=".",
)

run_job(Flow([elyte_interchange_job, production_flow]))


2024-04-02 15:13:28,211 INFO Started executing jobs locally
2024-04-02 15:13:28,217 INFO Starting job - generate_interchange (c1da9796-3b4d-4105-a385-2a0c7a15a5e3)
2024-04-02 15:13:31,042 INFO generate_interchange failed with exception:
Traceback (most recent call last):
  File "/Users/orioncohen/miniconda3/envs/atomate_emmet/lib/python3.11/site-packages/jobflow/managers/local.py", line 114, in _run_job
    response = job.run(store=store)
               ^^^^^^^^^^^^^^^^^^^^
  File "/Users/orioncohen/miniconda3/envs/atomate_emmet/lib/python3.11/site-packages/jobflow/core/job.py", line 583, in run
    response = function(*self.function_args, **self.function_kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/orioncohen/projects/development/atomate2/src/atomate2/classical_md/core.py", line 112, in generate_interchange
    mol_specs.append(create_mol_spec(**spec))
                     ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/orioncohen/projects/develop

RuntimeError: Flow did not finish running successfully